In [ ]:
from lmfit import Parameters
import matplotlib
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.signal import wiener, filtfilt, butter, gaussian, freqz
from scipy.ndimage import filters
from core.util.units import compute_dft

In [ ]:
def sineexponentialdecay_testing():
    # generation of data for testing
    x_axis = np.linspace(0, 100, 100)
    x_nice = np.linspace(x_axis[0], x_axis[-1], 1000)

    mod, params = fitlogic.make_sineexponentialdecay_model()
    print('Parameters of the model', mod.param_names,
          ' with the independet variable', mod.independent_vars)

    params['amplitude'].value = abs(1 + abs(np.random.normal(0,4)))
    params['frequency'].value = abs(0.01 + abs(np.random.normal(0,0.2)))
    params['phase'].value = abs(np.random.normal(0,2*np.pi))
    params['offset'].value = 12 + np.random.normal(0,5)
    params['lifetime'].value = abs(0 + abs(np.random.normal(0,70)))
    print('\n',
          'amplitude', params['amplitude'].value, '\n',
          'frequency', params['frequency'].value, '\n',
          'phase', params['phase'].value, '\n',
          'offset', params['offset'].value, '\n',
          'lifetime', params['lifetime'].value)

    data_noisy = (mod.eval(x=x_axis, params=params) + 0.5* np.random.normal(size=x_axis.shape))
    data = data_noisy
    offset = np.average(data)

    # level data
    data_level = data - offset

    # perform fourier transform with zeropadding to get higher resolution
    data_level_zeropaded = np.zeros(int(len(data_level) * 2))
    data_level_zeropaded[:len(data_level)] = data_level
    fourier = np.fft.fft(data_level_zeropaded)
    stepsize = x_axis[1] - x_axis[0]  # for frequency axis
    freq = np.fft.fftfreq(data_level_zeropaded.size, stepsize)
    fourier_power = (fourier * fourier.conj()).real

    plt.plot(freq[:int(len(freq) / 2)], fourier_power[:int(len(freq) / 2)], '-or', label='dft')
    plt.xlim(0, 0.5)
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    result = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_noisy,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    plt.plot(x_axis, data_noisy, 'o--b', label='noisy data')
    plt.plot(x_nice,mod.eval(x=x_nice, params=params),'-g', label='original')
    print(result.fit_report())
    plt.plot(x_axis, result.init_fit, '-y', linewidth=2.0, label='initial fit')
    plt.plot(x_axis, result.best_fit, '-r', linewidth=2.0, label='best fit')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    units = dict()
    units['frequency'] = 'GHz'
    units['phase'] = 'rad'
    units['amplitude']='arb. u.'

In [ ]:
sineexponentialdecay_testing()

In [ ]:

def two_sine_exp_decay_offset_testing():
    """ Testing procedure for the implemented two sine with exponential decay
        and offset fit. """

    x_axis = np.linspace(5, 600 ,200)
    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 1
    freq1 = 0.011

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 2
    freq2 = 0.01

    offset = 1
    lifetime = 100

    data = (ampl1 * np.sin(2*np.pi*freq1*x_axis + phase1)
            + ampl2 * np.sin(2*np.pi*freq2*x_axis + phase2)
           )*np.exp(-(x_axis/lifetime)) + offset
    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*0.6


    result1 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    data_sub = noisy_data - result1.best_fit
    result2 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub,
        estimator=fitlogic.estimate_sineexponentialdecay)

    mod, params = fitlogic.make_sinedoublewithexpdecay_model()

    # Fill the parameter dict:
    params['s1_amplitude'].set(value=result1.params['amplitude'].value)
    params['s1_frequency'].set(value=result1.params['frequency'].value)
    params['s1_phase'].set(value=result1.params['phase'].value)

    params['s2_amplitude'].set(value=result2.params['amplitude'].value)
    params['s2_frequency'].set(value=result2.params['frequency'].value)
    params['s2_phase'].set(value=result2.params['phase'].value)

    lifetime = (result1.params['lifetime'].value + result2.params['lifetime'].value)/2
    params['lifetime'].set(value=lifetime, min=2*(x_axis[1]-x_axis[0]))
    params['offset'].set(value=data.mean())

    result = mod.fit(noisy_data, x=x_axis, params=params)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print("freq1:", result.params['s1_frequency'].value)
    print("freq2:", result.params['s2_frequency'].value)
    print(result.fit_report())

In [ ]:
two_sine_exp_decay_offset_testing()

In [ ]:
def two_sine_exp_decay_offset_testing2():
    """ Testing procedure for the implemented two sine with offset fit. """

    x_axis = np.linspace(5, 600 ,200)
    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 0.11
    freq1 = 0.02

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 1
    freq2 = 0.01

    offset = 1
    lifetime = 100

    data = (ampl1 * np.sin(2*np.pi*freq1*x_axis + phase1)
            + ampl2 * np.sin(2*np.pi*freq2*x_axis + phase2)
           ) * np.exp(-(x_axis/lifetime)) + offset

    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*0.2

    result = fitlogic.make_sinedoublewithexpdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sinedoublewithexpdecay)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
two_sine_exp_decay_offset_testing2()

In [ ]:
def two_sine_two_exp_decay_offset_testing():
    """ Testing procedure for the implemented two sine two exponential decay
        with offset fit. """

    x_axis = np.linspace(5, 600 ,200)
    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 1
    freq1 = 0.02

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 3
    freq2 = 0.01

    offset = 1
    lifetime1 = 100
    lifetime2 = 100

    data = (
        ampl1 * np.sin(2*np.pi*freq1*x_axis +phase1)*np.exp(-(x_axis/lifetime1))
        + ampl2 * np.sin(2*np.pi*freq2*x_axis +phase2)*np.exp(-(x_axis/lifetime2))
        + offset)
    
    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*0.5


    result1 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    data_sub = noisy_data - result1.best_fit
    result2 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub,
        estimator=fitlogic.estimate_sineexponentialdecay)

    plt.figure()
    plt.plot(x_axis, data_sub,'-', label='sub')
    plt.plot(x_axis, noisy_data, '-', label='noisy_data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    mod, params = fitlogic.make_sinedoublewithtwoexpdecay_model()

    # Fill the parameter dict:
    params['e1_amplitude'].set(value=result1.params['amplitude'].value)
    params['e1_frequency'].set(value=result1.params['frequency'].value)
    params['e1_phase'].set(value=result1.params['phase'].value)
    params['e1_lifetime'].set(value=result1.params['lifetime'].value, min=2*(x_axis[1]-x_axis[0]))


    params['e2_amplitude'].set(value=result2.params['amplitude'].value)
    params['e2_frequency'].set(value=result2.params['frequency'].value)
    params['e2_phase'].set(value=result2.params['phase'].value)
    params['e2_lifetime'].set(value=result2.params['lifetime'].value, min=2*(x_axis[1]-x_axis[0]))

    params['offset'].set(value=data.mean())

    result = mod.fit(noisy_data, x=x_axis, params=params)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print("freq1:", result.params['e1_frequency'].value)
    print("freq2:", result.params['e2_frequency'].value)

    print(result.fit_report())

In [ ]:
two_sine_two_exp_decay_offset_testing()

In [ ]:
def two_sine_two_exp_decay_offset_testing2():
    """ Testing procedure for the implemented two sine with offset fit. """

    x_axis = np.linspace(5, 400 ,200)
    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 2
    freq1 = 0.02

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 1
    freq2 = 0.01

    offset = 1
    lifetime1 = 100
    lifetime2 = 200

    data = (
        ampl1 * np.sin(2*np.pi*freq1*x_axis +phase1) * np.exp(-(x_axis/lifetime1))
        + ampl2 * np.sin(2*np.pi*freq2*x_axis +phase2) * np.exp(-(x_axis/lifetime2))
        + offset)
    
    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*1

    result = fitlogic.make_sinedoublewithtwoexpdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sinedoublewithtwoexpdecay)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
two_sine_two_exp_decay_offset_testing2()

In [ ]:
def three_sine_exp_decay_offset_testing():
    """ Testing procedure for the estimator for a three sine with exponential
        decay and with offset fit. """

    x_axis = np.linspace(5, 300 ,200)

    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 3
    freq1 = 0.03

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 2
    freq2 = 0.01

    phase3 = np.random.uniform()*2*np.pi
    ampl3 = 1
    freq3 = 0.05

    lifetime = 100
    offset = 1.1

    data = (
        ampl1 * np.sin(2*np.pi*freq1*x_axis +phase1)
        + ampl2 * np.sin(2*np.pi*freq2*x_axis +phase2)
        + ampl3 * np.sin(2*np.pi*freq3*x_axis +phase3))*np.exp(-(x_axis/lifetime)) + offset

    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape) * 1

    x_dft1, y_dft1 = compute_dft(x_val=x_axis, y_val=noisy_data, zeropad_num=1)

    plt.figure()
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time (micro-s)')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    res1 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    
    data_sub1 = noisy_data - res1.best_fit
    x_dft2, y_dft2 = compute_dft(x_val=x_axis, y_val=data_sub1, zeropad_num=1)

    res2 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub1,
        estimator=fitlogic.estimate_sineexponentialdecay)

    data_sub2 = data_sub1 - res2.best_fit
    res3 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub2,
        estimator=fitlogic.estimate_sineexponentialdecay)

    x_dft3, y_dft3 = compute_dft(x_val=x_axis, y_val=data_sub2, zeropad_num=1)

    plt.figure()
    plt.plot(x_dft1, y_dft1, '-', label='noisy_data (3 peaks)')
    plt.plot(x_dft2, y_dft2, '-', label='noisy_data (2 peaks)')
    plt.plot(x_dft3, y_dft3, '-', label='noisy_data (1 peak)')
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    mod, params = fitlogic.make_sinetriplewithexpdecay_model()

    params['s1_amplitude'].set(value=res1.params['amplitude'].value)
    params['s1_frequency'].set(value=res1.params['frequency'].value)
    params['s1_phase'].set(value=res1.params['phase'].value)

    params['s2_amplitude'].set(value=res2.params['amplitude'].value)
    params['s2_frequency'].set(value=res2.params['frequency'].value)
    params['s2_phase'].set(value=res2.params['phase'].value)

    params['s3_amplitude'].set(value=res3.params['amplitude'].value)
    params['s3_frequency'].set(value=res3.params['frequency'].value)
    params['s3_phase'].set(value=res3.params['phase'].value)

    lifetime = (
        res1.params['lifetime'].value
        + res2.params['lifetime'].value
        + res3.params['lifetime'].value)/3
    
    params['lifetime'].set(value=lifetime, min=2*(x_axis[1]-x_axis[0]))
    params['offset'].set(value=data.mean())

    result = mod.fit(noisy_data, x=x_axis, params=params)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
three_sine_exp_decay_offset_testing()

In [ ]:
def three_sine_exp_decay_offset_testing2():
    """ Testing procedure for the implemented three sine with exponential
        decay and offset fit. """

    x_axis = np.linspace(5, 300 ,200)

    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 3
    freq1 = 0.03

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 2
    freq2 = 0.01

    phase3 = np.random.uniform()*2*np.pi
    ampl3 = 2
    freq3 = 0.05

    lifetime = 100
    offset = 1.1

    data = (
        ampl1 * np.sin(2*np.pi*freq1*x_axis + phase1)
        + ampl2 * np.sin(2*np.pi*freq2*x_axis + phase2)
        + ampl3 * np.sin(2*np.pi*freq3*x_axis + phase3))*np.exp(-(x_axis/lifetime)) + offset

    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*1

    result = fitlogic.make_sinetriplewithexpdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sinetriplewithexpdecay)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
three_sine_exp_decay_offset_testing2()

In [ ]:

def three_sine_three_exp_decay_offset_testing():
    """ Testing procedure for the estimator for a three sine with three
        exponential decays and with offset fit. """

    x_axis = np.linspace(5, 300 ,200)

    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 3
    freq1 = 0.03

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 2
    freq2 = 0.01

    phase3 = np.random.uniform()*2*np.pi
    ampl3 = 1
    freq3 = 0.05

    lifetime1 = 100
    lifetime2 = 150
    lifetime3 = 200
    offset = 1.1

    data = ( ampl1 * np.sin(2*np.pi*freq1*x_axis +phase1) * np.exp(-(x_axis/lifetime1))
           + ampl2 * np.sin(2*np.pi*freq2*x_axis +phase2) * np.exp(-(x_axis/lifetime2))
           + ampl3 * np.sin(2*np.pi*freq3*x_axis +phase3) * np.exp(-(x_axis/lifetime3))
           + offset)

    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*1

    x_dft1, y_dft1 = compute_dft(x_val=x_axis, y_val=noisy_data, zeropad_num=1)

    plt.figure()
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time (micro-s)')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    res1 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    data_sub1 = noisy_data - res1.best_fit
    x_dft2, y_dft2 = compute_dft(x_val=x_axis, y_val=data_sub1, zeropad_num=1)
    res2 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub1,
        estimator=fitlogic.estimate_sineexponentialdecay)
    
    data_sub2 = data_sub1 - res2.best_fit
    res3 = fitlogic.make_sineexponentialdecay_fit(
        x_axis=x_axis,
        data=data_sub2,
        estimator=fitlogic.estimate_sineexponentialdecay)

    x_dft3, y_dft3 = compute_dft(x_val=x_axis, y_val=data_sub2, zeropad_num=1)

    plt.figure()
    plt.plot(x_dft1, y_dft1, '-', label='noisy_data (3 peaks)')
    plt.plot(x_dft2, y_dft2, '-', label='noisy_data (2 peaks)')
    plt.plot(x_dft3, y_dft3, '-', label='noisy_data (1 peak)')
    plt.xlabel('Frequency (MHz)')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    mod, params = fitlogic.make_sinetriplewiththreeexpdecay_model()

    params['e1_amplitude'].set(value=res1.params['amplitude'].value)
    params['e1_frequency'].set(value=res1.params['frequency'].value)
    params['e1_phase'].set(value=res1.params['phase'].value)
    params['e1_lifetime'].set(value=res1.params['lifetime'].value,
                              min=2*(x_axis[1]-x_axis[0]))

    params['e2_amplitude'].set(value=res2.params['amplitude'].value)
    params['e2_frequency'].set(value=res2.params['frequency'].value)
    params['e2_phase'].set(value=res2.params['phase'].value)
    params['e2_lifetime'].set(value=res2.params['lifetime'].value,
                              min=2*(x_axis[1]-x_axis[0]))

    params['e3_amplitude'].set(value=res3.params['amplitude'].value)
    params['e3_frequency'].set(value=res3.params['frequency'].value)
    params['e3_phase'].set(value=res3.params['phase'].value)
    params['e3_lifetime'].set(value=res3.params['lifetime'].value,
                              min=2*(x_axis[1]-x_axis[0]))

    params['offset'].set(value=data.mean())

    result = mod.fit(noisy_data, x=x_axis, params=params)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
three_sine_three_exp_decay_offset_testing()

In [ ]:
def three_sine_three_exp_decay_offset_testing2():
    """ Testing procedure for the implemented three sine with three
        exponential decay and offset fit. """

    x_axis = np.linspace(5, 300 ,200)

    phase1 = np.random.uniform()*2*np.pi
    ampl1 = 3
    freq1 = 0.03

    phase2 = np.random.uniform()*2*np.pi
    ampl2 = 2
    freq2 = 0.01

    phase3 = np.random.uniform()*2*np.pi
    ampl3 = 1
    freq3 = 0.05

    lifetime1 = 120
    lifetime2 = 150
    lifetime3 = 50
    offset = 1.1

    data = ( ampl1 * np.sin(2*np.pi*freq1*x_axis +phase1) * np.exp(-(x_axis/lifetime1))
           + ampl2 * np.sin(2*np.pi*freq2*x_axis +phase2) * np.exp(-(x_axis/lifetime2))
           + ampl3 * np.sin(2*np.pi*freq3*x_axis +phase3) * np.exp(-(x_axis/lifetime3))
           + offset)

    noisy_data = data + data.mean() * np.random.normal(size=x_axis.shape)*1.2

    result = fitlogic.make_sinetriplewiththreeexpdecay_fit(
        x_axis=x_axis,
        data=noisy_data,
        estimator=fitlogic.estimate_sinetriplewiththreeexpdecay)

    plt.figure()
    plt.plot(x_axis, result.best_fit,'-', label='fit')
    plt.plot(x_axis, noisy_data, 'o--', label='noisy_data')
    plt.plot(x_axis, data,'-', label='ideal data')
    plt.xlabel('Time micro-s')
    plt.ylabel('signal')
    plt.legend(bbox_to_anchor=(0, 1.02, 1, .102), loc=3, ncol=2, mode="expand", borderaxespad=0)
    plt.show()

    print(result.fit_report())

In [ ]:
three_sine_three_exp_decay_offset_testing2()